En particular los NaNs están expresados de varias maneras diferentes y habria que identificar si hay otras cosas que no estén estandarizadas y estandarizarlas

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date
import re

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import files
uploaded = files.upload()

Saving set_de_datos_unificado_juzgado - set_de_datos_unificado.csv to set_de_datos_unificado_juzgado - set_de_datos_unificado.csv


In [3]:
import io
df = pd.read_csv(io.BytesIO(uploaded['set_de_datos_unificado_juzgado - set_de_datos_unificado.csv']))


In [4]:
df.head(3)

,N,NRO_REGISTRO,TOMO,FECHA_RESOLUCION,N_EXPTE_EJE,FIRMA,MATERIA,ART_INFRINGIDO,CODIGO_O_LEY,CONDUCTA,CONDUCTA_DESCRIPCION,VIOLENCIA_DE_GENERO,V_FISICA,V_PSIC,V_ECON,V_SEX,V_SOC,V_AMB,V_SIMB,FRASES_AGRESION,GENERO_ACUSADO/A,NACIONALIDAD_ACUSADO/A,EDAD_ACUSADO/A AL MOMENTO DEL HECHO,NIVEL_INSTRUCCION_ACUSADO/A,GENERO_DENUNCIANTE,NACIONALIDAD_DENUNCIANTE,EDAD_DENUNCIANTE_AL_MOMENTO_DEL_HECHO,NIVEL_INSTRUCCION_DENUNCIANTE,DOMICILIO_DENUNCIANTE,ASENTAMIENTO_O_VILLA,FRECUENCIA_EPISODIOS,RELACION_Y_TIPO_ENTRE_ACUSADO/A_Y_DENUNCIANTE,HIJOS/AS_EN_COMUN,MEDIDAD DE PROTECCION VIGENTES AL MOMENTO DEL HECHO,ZONA_DEL_HECHO,LUGAR_DEL_HECHO\n,ASENTAMIENTO_O_VILLA.1,FECHA_DEL_HECHO,FECHA_DE_INICIO_DEL_HECHO,FECHA_DE_FINALIZACIÓN_DEL_HECHO,TIPO_DE_RESOLUCION,OBJETO_DE_LA_RESOLUCION,DETALLE,DECISION,ORAL_ESCRITA,HORA_DE_INICIO,HORA_DE_CIERRE,LINK,MODELO_DESTACADO,DURACION,SI/NO_RECURRENTE,DECISION_CAMARA_DE_APELACIONES,N_REGISTRO_Y_TOMO_CAMARA,LINK_CAMARA,SI/NO_RECURRENTE.1,DECISION_DE_ADMISIBILIDAD_CAMARA,N_REGISTRO_Y_TOMO_CAMARA_1,LINK_CAMARA_1,QUEJA_Y_RECURRENTE,DECISION_DE_ADMISIBILIDAD_TSJ,Nº_REGISTRO_Y_TOMO_CAMARA_2,LINK_CAMARA_2,DECISION_DE_FONDO_TSJ,N_REGISTRO_Y_TOMO_TSJ,LINK_TSJ,RECURSO_EXTRAORDINARIO_Y_RECURRENTE,DECISION_CSJN,N_REGISTRO_Y_TOMO_CSJN,LINK_CSJN
0,1.0,NaN,NaN,1_8_2016,9498_16,Pablo_Casas,penal,1,ley_14346,proteccion_animal,simples,no,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,NaN,usuario_de_facebook,argentino,no_corresponde,s/d,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,no_corresponde,s/d,s/d,no,27_03_16,NaN,NaN,definitiva,prision_preventiva,conocimiento_personal,hace_lugar,escrita,NaN,NaN,https://drive.google.com/open?id=0B9wNhp3Gjjaz...,NaN,0:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,NaN,NaN,1_8_2016,9904_16,Pablo_Casas,penal,149bis,codigo_penal_de_la_nacion,amenazas,simples,si,no,si,no,no,no,no,si,me la vas a pagar,masculino,s/d,17,s/d,femenino,argentina,17,secundario_en_curso,zona_oeste,no,primera_vez,ex_pareja,no,no,zona_oeste,en_domicilio_particular,no,25_3_16,NaN,NaN,definitiva,archivo,convalida,hace_lugar,escrita,NaN,NaN,https://drive.google.com/open?id=0B9wNhp3Gjjaz...,NaN,0:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,NaN,NaN,2_8_2016,1313_16,Pablo_Casas,contravencional,52,codigo_contravencional,hostigamiento,agravado_violencia_de_genero,si,si,si,si,no,no,no,si,NaN,masculino,argentino,50,terciario_incompleto,femenino,argentina,23,s/d,zona_oeste,no,esporadico,ex_pareja,si,no,zona_oeste,en_domicilio_particular,no,24_7_14,NaN,NaN,interlocutoria,medida_cautelar,rebeldia,hace_lugar,escrita,NaN,NaN,https://drive.google.com/open?id=0B9wNhp3Gjjaz...,NaN,0:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Solo trabajo con Violencia de genero 
df_violencia = df[df["VIOLENCIA_DE_GENERO"] == "si"].copy()
df_violencia.reset_index(drop=True, inplace=True)
df_violencia.shape

(891, 69)

In [6]:
nulidad= df_violencia.isnull().sum()

print (nulidad)

N                                      209
NRO_REGISTRO                            29
TOMO                                    29
FECHA_RESOLUCION                         0
N_EXPTE_EJE                              0
                                      ... 
LINK_TSJ                               891
RECURSO_EXTRAORDINARIO_Y_RECURRENTE    891
DECISION_CSJN                          891
N_REGISTRO_Y_TOMO_CSJN                 891
LINK_CSJN                              891
Length: 69, dtype: int64


In [21]:
df_violencia.replace('', np.nan, inplace=True)
df_violencia.replace ('s/d', np.nan, inplace=True)
df_violencia.replace('no_corresponde', np.nan, inplace=True)
df_violencia["LINK"].replace("<", np.nan, inplace=True)

In [8]:
##df_violencia.drop(["MODELO_DESTACADO", "LINK_TSJ", 'RECURSO_EXTRAORDINARIO_Y_RECURRENTE', 'DECISION_CSJN', 'N_REGISTRO_Y_TOMO_CSJN','LINK_CSJN','N_REGISTRO_Y_TOMO_TSJ'], axis=1, inplace=True)
df_violencia.dropna(axis='columns', how='all', inplace=True)

Se eliminaron 7 columnas.

In [9]:
# forward-fill
df_violencia.N.fillna(method='ffill', inplace= True)
df_violencia.NRO_REGISTRO.fillna(method='ffill', inplace= True)
df_violencia.TOMO.fillna(method='ffill', inplace= True)
#entendemos que los que estan en blanco es porque refieren al mismo expediente que arriba pero respecto de delitos distintos. 
df_violencia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 62 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   N                                                    891 non-null    float64
 1   NRO_REGISTRO                                         864 non-null    object 
 2   TOMO                                                 864 non-null    float64
 3   FECHA_RESOLUCION                                     891 non-null    object 
 4   N_EXPTE_EJE                                          891 non-null    object 
 5   FIRMA                                                891 non-null    object 
 6   MATERIA                                              891 non-null    object 
 7   ART_INFRINGIDO                                       891 non-null    object 
 8   CODIGO_O_LEY                                         891 non-null    o

In [10]:
empty_vf = df_violencia.loc[df_violencia['V_FISICA'].isnull()]
empty_vf
#empty_ve= df_violencia.loc[df_violencia['V_ECON'].isnull()]
#empty_vs= df_violencia.loc[df_violencia['V_SEX'].isnull()]
#empty_vsoc= df_violencia.loc[df_violencia['V_SOC'].isnull()]
#empty_vamb= df_violencia.loc[df_violencia['V_AMBIENTAL'].isnull()]
#empty_simb= df_violencia.loc[df_violencia['V_SIMBOLICA'].isnull()]

,N,NRO_REGISTRO,TOMO,FECHA_RESOLUCION,N_EXPTE_EJE,FIRMA,MATERIA,ART_INFRINGIDO,CODIGO_O_LEY,CONDUCTA,CONDUCTA_DESCRIPCION,VIOLENCIA_DE_GENERO,V_FISICA,V_PSIC,V_ECON,V_SEX,V_SOC,V_AMB,V_SIMB,FRASES_AGRESION,GENERO_ACUSADO/A,NACIONALIDAD_ACUSADO/A,EDAD_ACUSADO/A AL MOMENTO DEL HECHO,NIVEL_INSTRUCCION_ACUSADO/A,GENERO_DENUNCIANTE,NACIONALIDAD_DENUNCIANTE,EDAD_DENUNCIANTE_AL_MOMENTO_DEL_HECHO,NIVEL_INSTRUCCION_DENUNCIANTE,DOMICILIO_DENUNCIANTE,ASENTAMIENTO_O_VILLA,FRECUENCIA_EPISODIOS,RELACION_Y_TIPO_ENTRE_ACUSADO/A_Y_DENUNCIANTE,HIJOS/AS_EN_COMUN,MEDIDAD DE PROTECCION VIGENTES AL MOMENTO DEL HECHO,ZONA_DEL_HECHO,LUGAR_DEL_HECHO\n,ASENTAMIENTO_O_VILLA.1,FECHA_DEL_HECHO,FECHA_DE_INICIO_DEL_HECHO,FECHA_DE_FINALIZACIÓN_DEL_HECHO,TIPO_DE_RESOLUCION,OBJETO_DE_LA_RESOLUCION,DETALLE,DECISION,ORAL_ESCRITA,HORA_DE_INICIO,HORA_DE_CIERRE,LINK,DURACION,SI/NO_RECURRENTE,DECISION_CAMARA_DE_APELACIONES,N_REGISTRO_Y_TOMO_CAMARA,LINK_CAMARA,SI/NO_RECURRENTE.1,DECISION_DE_ADMISIBILIDAD_CAMARA,N_REGISTRO_Y_TOMO_CAMARA_1,LINK_CAMARA_1,QUEJA_Y_RECURRENTE,DECISION_DE_ADMISIBILIDAD_TSJ,Nº_REGISTRO_Y_TOMO_CAMARA_2,LINK_CAMARA_2,DECISION_DE_FONDO_TSJ
733,2337.0,2462,19.0,30_10_2019,8796_17,Pablo_Casas,penal,149bis,codigo_penal_de_la_nacion,amenazas,simples,si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,masculino,boliviano,34,NaN,femenino,boliviana,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31_12_2016,NaN,NaN,interlocutoria,cuestion_de_turno,rechaza_competencia,hace_lugar,escrita,NaN,NaN,https://drive.google.com/open?id=1NkuV9Gvqh5dO...,0:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
734,2337.0,2462,19.0,30_10_2019,8796_17,Pablo_Casas,penal,183,codigo_penal_de_la_nacion,daños,NaN,si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,masculino,boliviano,34,NaN,femenino,boliviana,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31_12_2016,NaN,NaN,interlocutoria,cuestion_de_turno,rechaza_competencia,hace_lugar,escrita,NaN,NaN,https://drive.google.com/open?id=1NkuV9Gvqh5dO...,0:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
890,2747.0,2856,27.0,07_07_20,33076_18,Pablo_Casas,penal,149bis,codigo_penal_de_la_nacion,amenazas,simples,si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,definitiva,suspension_del_proceso_a_prueba,extincion_accion_sobreseimiento,hace_lugar,escrita,NaN,NaN,https://drive.google.com/file/d/1ui1dsbqgvwlom...,0:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
empty_ve= df_violencia.loc[df_violencia['V_ECON'].isnull()]
empty_ve
#empty_vs= df_violencia.loc[df_violencia['V_SEX'].isnull()]
#empty_vsoc= df_violencia.loc[df_violencia['V_SOC'].isnull()]
#empty_vamb= df_violencia.loc[df_violencia['V_AMB'].isnull()]
#empty_simb= df_violencia.loc[df_violencia['V_SIMB'].isnull()]

,N,NRO_REGISTRO,TOMO,FECHA_RESOLUCION,N_EXPTE_EJE,FIRMA,MATERIA,ART_INFRINGIDO,CODIGO_O_LEY,CONDUCTA,CONDUCTA_DESCRIPCION,VIOLENCIA_DE_GENERO,V_FISICA,V_PSIC,V_ECON,V_SEX,V_SOC,V_AMB,V_SIMB,FRASES_AGRESION,GENERO_ACUSADO/A,NACIONALIDAD_ACUSADO/A,EDAD_ACUSADO/A AL MOMENTO DEL HECHO,NIVEL_INSTRUCCION_ACUSADO/A,GENERO_DENUNCIANTE,NACIONALIDAD_DENUNCIANTE,EDAD_DENUNCIANTE_AL_MOMENTO_DEL_HECHO,NIVEL_INSTRUCCION_DENUNCIANTE,DOMICILIO_DENUNCIANTE,ASENTAMIENTO_O_VILLA,FRECUENCIA_EPISODIOS,RELACION_Y_TIPO_ENTRE_ACUSADO/A_Y_DENUNCIANTE,HIJOS/AS_EN_COMUN,MEDIDAD DE PROTECCION VIGENTES AL MOMENTO DEL HECHO,ZONA_DEL_HECHO,LUGAR_DEL_HECHO\n,ASENTAMIENTO_O_VILLA.1,FECHA_DEL_HECHO,FECHA_DE_INICIO_DEL_HECHO,FECHA_DE_FINALIZACIÓN_DEL_HECHO,TIPO_DE_RESOLUCION,OBJETO_DE_LA_RESOLUCION,DETALLE,DECISION,ORAL_ESCRITA,HORA_DE_INICIO,HORA_DE_CIERRE,LINK,DURACION,SI/NO_RECURRENTE,DECISION_CAMARA_DE_APELACIONES,N_REGISTRO_Y_TOMO_CAMARA,LINK_CAMARA,SI/NO_RECURRENTE.1,DECISION_DE_ADMISIBILIDAD_CAMARA,N_REGISTRO_Y_TOMO_CAMARA_1,LINK_CAMARA_1,QUEJA_Y_RECURRENTE,DECISION_DE_ADMISIBILIDAD_TSJ,Nº_REGISTRO_Y_TOMO_CAMARA_2,LINK_CAMARA_2,DECISION_DE_FONDO_TSJ
733,2337.0,2462,19.0,30_10_2019,8796_17,Pablo_Casas,penal,149bis,codigo_penal_de_la_nacion,amenazas,simples,si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,masculino,boliviano,34,NaN,femenino,boliviana,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31_12_2016,NaN,NaN,interlocutoria,cuestion_de_turno,rechaza_competencia,hace_lugar,escrita,NaN,NaN,https://drive.google.com/open?id=1NkuV9Gvqh5dO...,0:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
734,2337.0,2462,19.0,30_10_2019,8796_17,Pablo_Casas,penal,183,codigo_penal_de_la_nacion,daños,NaN,si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,masculino,boliviano,34,NaN,femenino,boliviana,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31_12_2016,NaN,NaN,interlocutoria,cuestion_de_turno,rechaza_competencia,hace_lugar,escrita,NaN,NaN,https://drive.google.com/open?id=1NkuV9Gvqh5dO...,0:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
776,2435.0,2555,20.0,9_12_2019,53421_19,Pablo_Casas,penal,90,codigo_penal_de_la_nacion,lesiones,graves,si,si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,masculino,paraguayo,NaN,NaN,femenino,NaN,NaN,NaN,NaN,NaN,NaN,pareja,NaN,NaN,provincia_de_buenos_aires,NaN,no,22_11_2019,NaN,NaN,interlocutoria,cuestion_de_turno,rechaza_competencia,hace_lugar,escrita,NaN,NaN,https://drive.google.com/open?id=1GdGvPYGBwWue...,0:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
890,2747.0,2856,27.0,07_07_20,33076_18,Pablo_Casas,penal,149bis,codigo_penal_de_la_nacion,amenazas,simples,si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,definitiva,suspension_del_proceso_a_prueba,extincion_accion_sobreseimiento,hace_lugar,escrita,NaN,NaN,https://drive.google.com/file/d/1ui1dsbqgvwlom...,0:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
empty_genero= df_violencia.loc[df_violencia['GENERO_ACUSADO/A'].isnull()]
empty_genero

,N,NRO_REGISTRO,TOMO,FECHA_RESOLUCION,N_EXPTE_EJE,FIRMA,MATERIA,ART_INFRINGIDO,CODIGO_O_LEY,CONDUCTA,CONDUCTA_DESCRIPCION,VIOLENCIA_DE_GENERO,V_FISICA,V_PSIC,V_ECON,V_SEX,V_SOC,V_AMB,V_SIMB,FRASES_AGRESION,GENERO_ACUSADO/A,NACIONALIDAD_ACUSADO/A,EDAD_ACUSADO/A AL MOMENTO DEL HECHO,NIVEL_INSTRUCCION_ACUSADO/A,GENERO_DENUNCIANTE,NACIONALIDAD_DENUNCIANTE,EDAD_DENUNCIANTE_AL_MOMENTO_DEL_HECHO,NIVEL_INSTRUCCION_DENUNCIANTE,DOMICILIO_DENUNCIANTE,ASENTAMIENTO_O_VILLA,FRECUENCIA_EPISODIOS,RELACION_Y_TIPO_ENTRE_ACUSADO/A_Y_DENUNCIANTE,HIJOS/AS_EN_COMUN,MEDIDAD DE PROTECCION VIGENTES AL MOMENTO DEL HECHO,ZONA_DEL_HECHO,LUGAR_DEL_HECHO\n,ASENTAMIENTO_O_VILLA.1,FECHA_DEL_HECHO,FECHA_DE_INICIO_DEL_HECHO,FECHA_DE_FINALIZACIÓN_DEL_HECHO,TIPO_DE_RESOLUCION,OBJETO_DE_LA_RESOLUCION,DETALLE,DECISION,ORAL_ESCRITA,HORA_DE_INICIO,HORA_DE_CIERRE,LINK,DURACION,SI/NO_RECURRENTE,DECISION_CAMARA_DE_APELACIONES,N_REGISTRO_Y_TOMO_CAMARA,LINK_CAMARA,SI/NO_RECURRENTE.1,DECISION_DE_ADMISIBILIDAD_CAMARA,N_REGISTRO_Y_TOMO_CAMARA_1,LINK_CAMARA_1,QUEJA_Y_RECURRENTE,DECISION_DE_ADMISIBILIDAD_TSJ,Nº_REGISTRO_Y_TOMO_CAMARA_2,LINK_CAMARA_2,DECISION_DE_FONDO_TSJ
890,2747.0,2856,27.0,07_07_20,33076_18,Pablo_Casas,penal,149bis,codigo_penal_de_la_nacion,amenazas,simples,si,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,definitiva,suspension_del_proceso_a_prueba,extincion_accion_sobreseimiento,hace_lugar,escrita,NaN,NaN,https://drive.google.com/file/d/1ui1dsbqgvwlom...,0:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
empty_nacionalidad= df_violencia.loc[df_violencia['NACIONALIDAD_ACUSADO/A'].isnull()]
print (len (empty_nacionalidad))

26


In [14]:
empty_edad= df_violencia.loc[df_violencia['EDAD_ACUSADO/A AL MOMENTO DEL HECHO'].isnull()]
print (len (empty_edad))

26


In [15]:
empty_instruccion= df_violencia.loc[df_violencia['NIVEL_INSTRUCCION_ACUSADO/A'].isnull()]
print (len (empty_instruccion))

156


In [16]:
empty_nac_den= df_violencia.loc[df_violencia['NACIONALIDAD_DENUNCIANTE'].isnull()]
print (len (empty_nac_den))

20


In [17]:
empty_edad_den= df_violencia.loc[df_violencia['EDAD_DENUNCIANTE_AL_MOMENTO_DEL_HECHO'].isnull()]
print (len (empty_edad_den))

158
